## Юнит 9. Профориентация в Data Science
### Skillfactory: DSPR-19
### Проект 7. Возьмёте Бэтмобиль? 

### 1. Чем займёмся в этом проекте?
→ Поздравляем! Вы приступаете к выполнению финального проекта блока Профориентация, в котором вы сможете на практике поработать с задачами из всех трёх направлений — ML, CV и NLP.

За основу возьмем решение, которое нам уже известно, — построенную на табличных данных модель «Предсказание стоимости автомобиля» — и добавим туда модели Deep Learning, которые будут распознавать изображения и обрабатывать текст.

За последние несколько месяцев работы в компании «Старый друг» вы разработали две модели, которые уже успешно крутятся в продакшне и приносят деньги компании и радость сотрудникам (по словам Анны из отдела продаж).

→ Осталось довести свою работу до логичного завершения и, обогатив датасет текстовыми данными из объявлений о продаже, свести все наши модели в единое решение — Multi-inputs сеть. Приступим?

### КАК БУДЕМ РЕШАТЬ ПРОЕКТ?

### ПОСТРОИМ БАЗОВОЕ РЕШЕНИЕ:

- Шаг 1. Построим «наивную» ML-модель на табличных данных, которая предсказывает цену по модели и году выпуска. Это позволит нам задать направление для дальнейших экспериментов. Впоследствии на этом этапе вы можете доработать вашу модель из проекта «Выбираем авто выгодно». 
- Шаг 2. Обработаем и отнормируем признаки и сделаем первую модель на основе градиентного бустинга с помощью CatBoost.
- Шаг 3. Решим эту же задачу с помощью DL (модель NN Tabular) и сравним результаты.
- Шаг 4. Добавим текстовые данные (NLP) и сделаем Multi-Input нейронную сеть для анализа и табличных данных, и текста одновременно.
- Шаг 5. Добавим обработку изображений в Multi-Input нейронную сеть.
- Шаг 6. Осуществим ансамблирование градиентного бустинга и нейронной сети (усредним их предсказания).

### САМОСТОЯТЕЛЬНАЯ ЧАСТЬ

- Ознакомьтесь с критериями оценивания проекта.
- Следуйте нашим рекомендациям, чтобы построить самую лучшую модель.
- Задавайте вопросы одногруппникам и менторам в Slack.
- Загрузите итоговое решение на kaggle и добавьте описание проекта на git.  

На выполнение проекта отводится: две недели.

### ЧТО ВЫ ПОЛУЧИТЕ В РЕЗУЛЬТАТЕ ПРОЕКТА?

- Потренируетесь обрабатывать данные и оптимизировать NLP-модели.
- Пройдетесь по всему циклу разработки комплексной Multi-Inputs модели (от обработки данных до внедрения в продакшн).
- Увидите, как использование blend-решений влияет на целевые метрики.
- Примерите на себя роль специалиста каждого из трёх треков второго года обучения.



— Возьмете Бэтмобиль?  
— Нет. Слишком заметно.  
— Ну тогда возьмите Ламборгини. Это менее заметно.

### 2. Начнём с простой модели

Начнём с разработки baseline нашего решения и посмотрим, как происходит обращение с входящими данными и что нужно получить на выходе. 

По ссылке вы обнаружите baseline. 

СКАЧАТЬ BASELINE (https://github.com/luhakv/cv_ml_nlp/blob/master/baseline-sf-dst-car-price-part2-v6.ipynb)

Итак, наша задача — предсказать стоимость автомобиля по его характеристикам, текстовому описанию или картинке. 

Начнём знакомиться с baseline-решением. Как видим, данные не сильно отличаются от прежних:

bodyType — категориальный  
brand —  категориальный   
color —  категориальный  
description —  текстовый  
engineDisplacement — числовой, представленный как текст  
enginePower — числовой, представленный как текст  
fuelType — категориальный  
mileage —  числовой   
modelDate — числовой  
model_info — категориальный  
name — категориальный, желательно сократить размерность  
numberOfDoors — категориальный  
price — числовой, целевой  
productionDate — числовой  
sell_id — изображение (файл доступен по адресу, основанному на sell_id)  
vehicleConfiguration — не используется (комбинация других столбцов)  
vehicleTransmission — категориальный  
Владельцы — категориальный  
Владение — числовой, представленный как текст  
ПТС — категориальный  
Привод — категориальный  
Руль — категориальный  


Однако кое-что, скорее всего, придётся переделать, поскольку есть дополнительные столбцы. 

→ Переходим к построению самой простой «наивной» модели, которая предсказывает среднюю цену по модели автомобиля и году выпуска. Начинать решение всегда стоит именно с простой модели, чтобы понимать, куда вы движетесь — к улучшению или к ухудшению решения. Это сэкономит вам множество сил и ресурсов. 

Наша модель предсказывает среднее значение по стоимости относительно модели и года выпуска.



In [ ]:
# Наивная модель
predicts = []
for index, row in pd.DataFrame(data_test[['model_info', 'productionDate']]).iterrows():
    query = f"model_info == '{row[0]}' and productionDate == '{row[1]}'"
    predicts.append(data_train.query(query)['price'].median())

# заполним не найденные совпадения
predicts = pd.DataFrame(predicts)
predicts = predicts.fillna(predicts.median())

# округлим
predicts = (predicts // 1000) * 1000

#оцениваем точность
print(f"Точность наивной модели по метрике MAPE: {(mape(data_test['price'], predicts.values[:, 0]))*100:0.2f}%")

### 3. EDA и модель на основе градиентного бустинга

На этом этапе нам важно посмотреть на распределение числовых признаков, опуская категориальные, поскольку задача этого шага — посмотреть, подходят ли данные для загрузки в сеть. 




In [ ]:
#посмотрим, как выглядят распределения числовых признаков
def visualize_distributions(titles_values_dict):
  columns = min(3, len(titles_values_dict))
  rows = (len(titles_values_dict) - 1) // columns + 1
  fig = plt.figure(figsize = (columns * 6, rows * 4))
  for i, (title, values) in enumerate(titles_values_dict.items()):
    hist, bins = np.histogram(values, bins = 20)
    ax = fig.add_subplot(rows, columns, i + 1)
    ax.bar(bins[:-1], hist, width = (bins[1] - bins[0]) * 0.7)
    ax.set_title(title)
  plt.show()

visualize_distributions({
    'mileage': train['mileage'].dropna(),
    'modelDate': train['modelDate'].dropna(),
    'productionDate': train['productionDate'].dropna()
})

Распределение признаков ненормальное, поэтому их необходимо нормировать. 

Разбиваем на категориальные признаки и числовые:

In [ ]:
#используем все текстовые признаки как категориальные без предобработки
categorical_features = ['bodyType', 'brand', 'color', 'engineDisplacement', 'enginePower', 'fuelType', 'model_info', 'name',
  'numberOfDoors', 'vehicleTransmission', 'Владельцы', 'Владение', 'ПТС', 'Привод', 'Руль']

#используем все числовые признаки
numerical_features = ['mileage', 'modelDate', 'productionDate']

Объединяем тренировочный и тестовый сеты:



In [ ]:
train['sample'] = 1 # помечаем где у нас трейн
test['sample'] = 0 # помечаем где у нас тест
test['price'] = 0 # в тесте у нас нет значения price, мы его должны предсказать, по этому пока просто заполняем нулями

data = test.append(train, sort=False).reset_index(drop=True) # объединяем
print(train.shape, test.shape, data.shape)

Выполняем предобработку данных: удаляем лишнее, заполняем пропуски, если они есть, выполняем нормирование. На CatBoost это не должно повлиять. Наконец, выполняем Label Encoding или One-Hot Encoding.



In [ ]:
def preproc_data(df_input):
    '''includes several functions to pre-process the predictor data.'''
    
    df_output = df_input.copy()
    
    # ################### 1. Предобработка ############################################################## 
    # убираем не нужные для модели признаки
    df_output.drop(['description','sell_id',], axis = 1, inplace=True)
    
    
    # ################### Numerical Features ############################################################## 
    # Далее заполняем пропуски
    for column in numerical_features:
        df_output[column].fillna(df_output[column].median(), inplace=True)
    # тут ваш код по обработке NAN
    # ....
    
    # Нормализация данных
    scaler = MinMaxScaler()
    for column in numerical_features:
        df_output[column] = scaler.fit_transform(df_output[[column]])[:,0]
    
    
    
    # ################### Categorical Features ############################################################## 
    # Label Encoding
    for column in categorical_features:
        df_output[column] = df_output[column].astype('category').cat.codes
        
    # One-Hot Encoding: в pandas есть готовая функция - get_dummies.
    df_output = pd.get_dummies(df_output, columns=categorical_features, dummy_na=False)
    # убираем признаки которые еще не успели обработать, 
    df_output.drop(['vehicleConfiguration'], axis = 1, inplace=True)
    
    return df_output


Запускаем и смотрим на чистые данные. 



In [ ]:
df_preproc = preproc_data(data)
df_preproc.sample(10)

Мы получили целевую переменную price, дату производства, пробег, столбец с указанием принадлежности к тренировочному или тестовому наборам.   

Затем выделяем тестовую часть:



In [ ]:
train_data = df_preproc.query('sample == 1').drop(['sample'], axis=1)
test_data = df_preproc.query('sample == 0').drop(['sample'], axis=1)

y = train_data.price.values     # наш таргет
X = train_data.drop(['price'], axis=1)
X_sub = test_data.drop(['price'], axis=1)

Теперь мы можем запустить уже не наивную модель, а построить модель на основе градиентного бустинга при помощи CatBoost. 



In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.15, shuffle=True, random_state=RANDOM_SEED)


In [ ]:
model = CatBoostRegressor(iterations = 5000,
                          #depth=10,
                          #learning_rate = 0.5,
                          random_seed = RANDOM_SEED,
                          eval_metric='MAPE',
                          custom_metric=['RMSE', 'MAE'],
                          od_wait=500,
                          #task_type='GPU',
                         )
model.fit(X_train, y_train,
         eval_set=(X_test, y_test),
         verbose_eval=100,
         use_best_model=True,
         #plot=True
         )

Проверяем MAPE у тестовой части: 



In [ ]:
test_predict_catboost = model.predict(X_test)
print(f"TEST mape: {(mape(y_test, test_predict_catboost))*100:0.2f}%")

Мы получили MAPE 13.23%. Это неплохой результат, поскольку CatBoost хорошо работает с небольшим количеством данных и множеством категориальных признаков.

Сохраняем submission: 

In [ ]:
sub_predict_catboost = model.predict(X_sub)
sample_submission['price'] = sub_predict_catboost
sample_submission.to_csv('catboost_submission.csv', index=False)